In [4]:
import math
import numpy as np

row = 10 
column = 10 
def load_parameters(file_name):
    g_w = []
    t_l = []
    n_d = []
    with open(file_name) as f:
        file=f.readlines()
        gw=file[2:12]
        tl=file[16:20]
        nd=file[24:]
        for a in gw:
            ad=a.strip('\n').split(' ')
            g_w1=[]
            for a1 in ad:
                a1=int(a1)
                g_w1.append(a1)
            g_w.append(g_w1)
        for b in tl:
            t_l.append([int(b.strip('\n').split(' ')[2]),int(b.strip('\n').split(' ')[3])])
        for c in nd:
            n_d1=[]
            cd=c.strip('\n').split(' ')
            while '' in cd:
                 cd.remove('')
            for c1 in cd: 
                c1=float(c1)
                n_d1.append(c1)
            n_d.append(n_d1)
    return g_w, t_l, n_d
    
def NextStepProbability(x, y, grid_world):
      #相邻的两个cell，在 cell 1 情况下走到 cell 2 的概率
    o = 0#统计cell 1可走的方向，1/方向数=在 cell 1 情况下走到 cell 2 的概率 
    if((y-1 >=0) and (grid_world[x][y-1]==1)):
         # Left cell is available.
        o += 1
    else: pass    
    
    if ((y+1 < column) and (grid_world[x][y+1]==1)):
         # Right cell is available.
        o += 1
    else: pass 
            
    if ((x-1 >=0) and (grid_world[x-1][y]==1)):
         # Up cell is available.
        o += 1
    else: pass

    if ((x+1 < row) and (grid_world[x+1][y]==1)):
        # Down cell is available.
        o += 1
    else: pass 
            
    if (o > 0):
        return 1/o    
    else:
        return 0  
      
    
def Probability(Upx, Upy, x, y, grid_world):
    #在 cell 1 情况下走到 cell 2 的概率 
    #The ith row has a x coordinate of i, and the jth column has a y coordinate of j on the grid world.
    if((x-1 == Upx) and (y == Upy)): # Are (x, y) next to (under) the (x_1, y_1)?
        return NextStepProbability(Upx, Upy, grid_world)
    elif ((x+1 == Upx) and (y == Upy)): # Are (x, y) next to (at top of) the (x_1, y_1)?
        return NextStepProbability(Upx, Upy, grid_world)
    elif ((y-1 == Upy) and (x == Upx)): # Are (x, y) next to (at right of) the (x_1, y_1)?
        return NextStepProbability(Upx, Upy, grid_world)
    elif ((y+1 == Upy) and (x == Upx)): # Are (x, y) next to (at left of) the (x_1, y_1)?
        return NextStepProbability(Upx, Upy, grid_world)
    else:# cell 1 == cell 2，或者 很远不可能一步走到的情况
        return 0    

def CalculateDistance(towerlocation, grid_world):
    #DISTANCE_MATRIX: Store distance(observation) probability of each cell for each tower . 
    #=> [[tower1],...,[tower4]], 
    #[tower1]=[cell1[Probability of distance0, ..., Probability of distanceN]] stateN[Probability of evidence1, ...,
    number_precistion = 1
    distance_matrix = []#Distance matrix
    observations = [] #Observations
    max_distance = 0
    min_distance = 0  #Due to noise, the minimum distance return to robot will be 0.7*d，最小噪声距离
    max_distance = 0 #Due to noise, the minimum distance return to robot will be 1.3*d，最大噪声距离
    unit = 0.1**number_precistion#小数点到 0.1
    tower_x, tower_y = 0, 0  #the coordinate of tower
    row, column = np.array(grid_world).shape
    max_distance = math.sqrt((row-1)**2 + (column-1)**2)    #最大实际距离
    min_noisy = 0.7 #interval [0.7d,1.3d]
    max_noisy = 1.3

    for i in range(0, int(max_distance * max_noisy/unit)+1):
        observations.extend([round(i*unit,number_precistion)])
    
    for tower in towerlocation:
        prob_matrix = []
        for cell in STATES:
            tower_x = tower[0]
            tower_y = tower[1]
            distance = math.sqrt((AvailableCOORDINATION[cell][0]-tower_x)**2+(AvailableCOORDINATION[cell][1]-tower_y)**2)
            min_distance = round(min_noisy * distance, number_precistion)
            max_distance = round(max_noisy * distance, number_precistion)
            probability = 1/((max_distance - min_distance + unit)/unit)
            prob_matrix.append([])
            for i in observations:
                if (min_distance <= i and i <= max_distance):
                    prob_matrix[cell-1].extend([probability])
                else:
                    prob_matrix[cell-1].extend([0])                    
        distance_matrix.append(prob_matrix)
    return distance_matrix, observations


def CPT(grid_world):
    #Calculate the conditional probability table
    #  Compute the initial state and transition matrix from cell1 to cell2.
    #  The ith row has a x coordinate of i, and the jth column has a y coordinate of j on the grid world.
    states = []#states的标号 从1到87
    transition_matrix = []#transition matrix
    initial_probability = [] #initial probability
    total_states = 0#cell总数
    valid_states = 0#有多少个可用的cell
    x = 0#一共几行
    y = 0#每行几个cell=一共几列
    upx = 0
    upy = 0
    validcoord = {}#可用cell的坐标
    # Match the state to x, y coordination. 
    for xx in grid_world:  #row is x
        for yy in xx:  #column is y
            if yy == 1:
                validcoord[(valid_states+1)] =[x, y]
                states.extend([valid_states+1])#From state1
                valid_states += 1 
                total_states += 1
                y += 1
            else:
                total_states +=1
                y += 1
        y = 0# 下一个循环是新的一行计数
        x += 1
        
    # Compute probability for S0 to S1
    # init_p：1/87=0.01149425
    
    init_p = 1/valid_states#Same probability for each valid cell.
    #所有cell的初始化的概率矩阵：1*87
    initial_probability = np.ones((1, valid_states))*init_p
    #print('total cells=%d, valid cells=%d, initial probability=%f' % (total_states, valid_states, init_p))
     # Compute probability for S1 for X2. 转换矩阵
    #87*87
    transition_matrix = np.zeros((valid_states, valid_states))
    # Define transition matrix
    # _tm = [[S0], [S1], [S2]....[Sn]] to represent the probability of robot locate in each cell if the robot locate in cell n at time (t-1).
    # [Sn]=[x0, x1, x2,....xn] to represent the probability of robot locate in each cell if the robot previous locate in cell n at time (t). 
    
    for xx in range (valid_states):#xt-1=f1, f2,...,f87
        for yy in range (valid_states): #Calculate from St-1 to St. xt=f1, f2,...,f87
            #从1到87每一个的坐标，t时间的cell 1
            upx, upy = validcoord[xx+1]
            # 计算上一次在cell 1（_x_1, _y_1），下一步到cell 2 （_x, _y）的概率 
            x, y = validcoord[yy+1]
            transition_matrix[xx][yy] = Probability(upx, upy, x, y, grid_world)
            
    return states, initial_probability, transition_matrix, validcoord

def viterbi(observation_seq, distance_matrix,observations,states,initial_prob,transition_matrix):
    trace_back_states=[]
    steps = 0
    states_max_probability = []# The highest probability of each state at time_step t
    previous_states = []# The previous state link to current state
    states_sequence = []# The out put of state sequences.
    emission_probability = []#The integrated emission probability from multiple emission probability tables.
    if (np.array(observation_seq).ndim == 1): #add sequence from one to two dimensions
        observation_seq = [observation_seq]
    else:
        pass
    if (np.array(distance_matrix).ndim ==1):
            distance_matrix = [distance_matrix]
    else:
        pass
        
    steps = len(observation_seq)
    if (steps == 0 or len(initial_prob[0]) == 0 ):#If input observation sequence is zero length.
        return states_sequence
    else:
        pass
               
    for step in range(steps):
        emission_probability = eliminate_evidences(observation_seq, step,distance_matrix,observations,states)#Get the probability of state from given observation sequence of evidences
        if step == 0: #First step
            states_max_probability = initial_prob * emission_probability            
        else:
            states_max_probability, previous_states = max_probability_state(states_max_probability * transition_matrix) 
            states_max_probability = states_max_probability * emission_probability
            trace_back_states.append(previous_states)
                
    states_sequence.extend([max_prob(states_max_probability)])#Record down the highest probability of state in last time step.
     #back track to each time steps for the highest probability of state in each time step.
    trace_back_states.reverse()
    for previous_state_list in trace_back_states:
        states_sequence.extend([previous_state_list[states_sequence[-1]]])
    states_sequence.reverse()    
    return states_sequence
    
def eliminate_evidences(observation_seq, step,distance_matrix,observations,states):
     #To support multiple evidences
        # Return the probability of combination of all evidences.
        # _tmp_state_prob: List of probability list for every state at time t for all evidences.
        #    =>[Evidence1[probability of state1, ..., probability of stateN], ..., EvidenceN[probability of state1, ..., probability of stateN]]
        # _state_prob: List of consolidate probability for every state at time t.
        #    =>[probability of state1, ..., probability of stateN]
    state_probability = np.ones((1, len(states)))
    tmp_state_probability = []
    
    for i, ev in enumerate(observation_seq[step]):  #evidence i at step of time
        tmp_state_probability.append([])
        for j, st in enumerate(states): # state j
            for k, observation in enumerate(observations): #observation k
                if(ev == observation): #If evidence value is match to the label of observations.
                    tmp_state_probability[i].extend([distance_matrix[i][j][k]]) #Get the probability from emission probability matrix.
                    break
                else:
                    pass

    for tmp_probability in tmp_state_probability:#Consolidate evidence from multiple to one
        state_probability *= tmp_probability 
    return state_probability

def max_probability_state(states_probability):
     # To get maximum probability of state list at time t
        # It will return max_prob list for each state at time t and its previous state at time t-1 
        # For tie breaking, please always prefer the one with a smaller x coordinate, and a smaller y coordinate if the x coordinates are equal.
    max_probability = 0
    max_previous_state = 0
    state_len = len(STATES)
    max_probability_list = []
    max_previous_state_list = []
    for current_state in range(state_len):#Review each column from row1, ..., rowN.
        max_probability = 0
        previous_state = 0
        for previous_state in range(state_len):
            probability = states_probability[current_state][previous_state]
            if (probability > max_probability):
                max_probability = probability
                max_previous_state = previous_state
            else:
                pass 
        max_probability_list.extend([max_probability])  
        max_previous_state_list.extend([max_previous_state]) 
    return max_probability_list, max_previous_state_list

def max_prob(states_probability):
     # To get maximum probability of state at time t
        # For tie breaking, please always prefer the one with a smaller x coordinate, and a smaller y coordinate if the x coordinates are equal.
    state = 0        
    max_probability = 0
    for st, probability in enumerate(states_probability[0]):#Review row0 first, then each column. row1, column 0 ~ 9...etc.
        if (probability > max_probability):
            max_probability = probability
            state = st
        else:
            pass    
    return state
    
def state2coordinate(sequence_list, labels_dictionary):
    sequence_list = sequence_list
    label_sequence = []
    
    for sequence in sequence_list:
        label_sequence.extend([labels_dictionary[sequence+1]]) #starting state from 1 rather than 0.
    
    return label_sequence

           

GRID_WORLD, TOWER_LOCATION, NOISY_DISTANCES = load_parameters('hmm-data.txt')
#print ('NOISY_DISTANCES:', NOISY_DISTANCES)
STATES, INITIAL_PROB, TRANSITION_MATRIX, AvailableCOORDINATION = CPT(GRID_WORLD)
#print('STATES:', STATES)
#print('AvailableCOORDINATION:', AvailableCOORDINATION)
DISTANCE_MATRIX, OBSERVATIONS = CalculateDistance(TOWER_LOCATION, GRID_WORLD)
#print('OBSERVATIONS:', OBSERVATIONS)
state_sequence = viterbi(NOISY_DISTANCES,DISTANCE_MATRIX,OBSERVATIONS,STATES,INITIAL_PROB,TRANSITION_MATRIX)
#print ('State sequence:', [x+1 for x in state_sequence]) #Because we define the first state counts from 1
print ('Coordination sequence:', state2coordinate(state_sequence, AvailableCOORDINATION))


Coordination sequence: [[5, 3], [6, 3], [7, 3], [7, 2], [7, 1], [7, 2], [7, 1], [6, 1], [5, 1], [4, 1], [3, 1]]
